In [1]:
import tweepy
from azure.eventhub.aio import EventHubProducerClient
from azure.eventhub import EventData
from tweepy.auth import OAuthHandler
from tweepy import Stream
from tweepy import StreamListener
import json
import asyncio

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
class sendData():
    
    def __init__(self):
        
        self.ckey='CONSUMER_KEY'
        self.csecret='CONSUMER_SECRET'
        self.akey='ACCESS_KEY'
        self.asecret='ACCESS_SECRET'
        print("start")
        self.connect_to_twitter_api()
        
            
    def connect_to_twitter_api(self):
        
        auth = OAuthHandler(self.ckey,self.csecret)
        auth.set_access_token(self.akey,self.asecret)
        print("connected to twitter")
        self.call_twitter_api(auth)

        
    def call_twitter_api(self,auth):
        
        print("call twitter api")
        twitter_stream = Stream(auth,Tweet_listner())
        twitter_stream.filter(track=["trump"])
        
    def get_sentiment_from_text_blob(self):
        pass

In [4]:
class Tweet_listner(StreamListener):
    
    async def run_event(self,text):
        print("run")
        producer = EventHubProducerClient.from_connection_string(conn_str="CONNECTION_STRING",eventhub_name="EVENTHUB_NAME")
        print(producer)
        
        async with producer:
            # Create a batch.
            print(text)
            event_data_batch = await producer.create_batch()
            event_data_batch.add(EventData(text))
            await producer.send_batch(event_data_batch)
        
    def on_data(self,data):
        
        loop = asyncio.get_event_loop()
        
        tweets=json.loads(data)
        
        refined_tweet={"location":tweets["user"]["location"],"tweet":tweets["text"]}
        
        print(refined_tweet)
        loop.run_until_complete(self.run_event(json.dumps(refined_tweet)))
        
        return(True)
    
    def on_error(self,status):
        pass

In [5]:
if __name__ =="__main__":
    Send_data = sendData()

start
connected to twitter
call twitter api
{'location': None, 'tweet': '@BuckSexton What about your king, trump?'}
run
{"location": null, "tweet": "@BuckSexton What about your king, trump?"}
{'location': 'The Great Pacific Northwest', 'tweet': "RT @StewardshipAmer: Well, whadda ya know; no one ever would have thought that cloth and surgical masks don't stop viruses. Ya think this d…"}
run
{"location": "The Great Pacific Northwest", "tweet": "RT @StewardshipAmer: Well, whadda ya know; no one ever would have thought that cloth and surgical masks don't stop viruses. Ya think this d\u2026"}
{'location': 'Chandigarh', 'tweet': "RT @johncardillo: Meanwhile, last September:\n\nTrump Says He's Not a Fan of Meghan Markle, Wishes Prince Harry Luck  https://t.co/U1x3a6V6DL"}
run
{"location": "Chandigarh", "tweet": "RT @johncardillo: Meanwhile, last September:\n\nTrump Says He's Not a Fan of Meghan Markle, Wishes Prince Harry Luck  https://t.co/U1x3a6V6DL"}
{'location': 'United Kingdom', 'tweet':

ProtocolError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))